In [1]:
import os, glob, gzip
import pandas as pd
from tqdm import tqdm
import json
from datetime import datetime

pd.set_option('display.max_columns', None)
                        
import warnings
warnings.filterwarnings('ignore')

> #### Iterate through the directory recursively and extract the tar.gz files

In [2]:
def get_gzip_file_paths(root_dir):
    
    '''
        Gzipped files are where the clickthrough data is stored.
        Collect all the gzip paths recursively starting the traversal
        from the root_directory
        
        @return List of gzips (clickthrough files)
    '''
    
    gzip_file_paths = []
    for subdir, dirs, files in tqdm(os.walk(root_dir)):

        for file in files:
            filepath = os.path.join(subdir, file)
            if filepath.endswith(".gz"):
                gzip_file_paths.append(filepath)
                
    return gzip_file_paths

In [3]:
def parse_clickthrough_gzip(path):
    
    try:
        df = pd.read_json(path, lines=True, compression='gzip')
        df["filepath"] = path.split("clickstream_data_2016")[1][1:]
        df["request_keys"] = df["request"].apply(lambda x: x.keys())
        df["server_request_keys"] = df["server_request"].apply(lambda x: x.keys())

        request_attributes_all = [list(x) for x in df["request_keys"].tolist()]
        request_attributes = list(set([item for sublist in request_attributes_all for item in sublist]))
        for attribute in request_attributes:
            df[attribute] = df["request"].apply(lambda x: x[attribute] if attribute in x else None)

        serv_request_attributes_all = [list(x) for x in df["server_request_keys"].tolist()]
        serv_request_attributes = list(set([item for sublist in serv_request_attributes_all for item in sublist]))
        for attribute in serv_request_attributes:
            df[attribute] = df["server_request"].apply(lambda x: x[attribute] if attribute in x else None)


        del df["request_keys"]
        del df["server_request_keys"]
        del df["request"]
        del df["server_request"]
        return df
    except:
        return None
    
    
def write_failed_filelist(failed_files, swap_dir, failed_filename):
    '''
        Validation 1: Reporting any files that have failed during reading
        
        Make a note of the list of gzip files that failed
        while reading
    '''
    if len(failed_files) > 0:
        failed_files = ["\n"+f for f in failed_files]
        with open(os.path.join(swap_dir, failed_filename), "w") as f:
            f.writelines(failed_files)    
    
        

def read_gzip_files(gzip_file_paths, swap_path, failed_filename):
    
    '''
    Read the gzipped files into a dataframe and 
    construct the clickthrough attributes.
    
    @return List of dataframes with clickthrough attributes, 
            one for each file
    @return List of dataframes which have
    '''

    json_data = list(map(parse_clickthrough_gzip, gzip_file_paths))
    failed_files = [gzip_file_paths[i] for i, df in enumerate(json_data) if df is None]
    
    write_failed_filelist(failed_files, swap_path, failed_filename)
    
    return json_data

### Validation

In [4]:
def get_all_headers(dfs):
    
    '''
    Most dataframes might not have the full set of keys.
    Get every key which was present in request/ server_request.
    '''
    
    headers = [x.columns.tolist() for x in dfs]
    headers = list(set([item for sublist in headers for item in sublist]))
    return headers


def format_clickthrough_dataframes(df, headers_to_include):

    
    missing_cols = [x for x in headers_to_include if x not in df.columns.tolist()]
    if len(missing_cols) > 0:
        for col in missing_cols:
            df[col] = None
    df = df[headers_to_include]
    return df

In [5]:
def make_readable_date(df):
    
    df["request_datetime"] = df["request_unixtime"].apply(lambda x: datetime.fromtimestamp(x))

### Variables to run the code
---

In [6]:
# Ideally this has to be read in from a configuration file. 
# However, reading from a variable for now.
root_directory = '/home/sree/code/apptio-data/clickstream_data_2016'
swap_directory = "swap"
start_time = str(datetime.now())[:16]
failed_file_name = "failed_" + start_time + ".txt"

### Processing
---

In [7]:
gzip_paths = get_gzip_file_paths(root_directory)
clickthrough_dfs = read_gzip_files(gzip_paths[:2], swap_directory, failed_file_name)

# Using all headers for now. Later depending on the usecase, we can use a subset of these
# columns for our analysis
headers = get_all_headers(clickthrough_dfs)
clickthrough_dfs = [format_clickthrough_dataframes(df, headers) for df in clickthrough_dfs]
input_data = pd.concat(clickthrough_dfs)

make_readable_date(input_data)
input_data.head()

205it [00:00, 30904.76it/s]


,requestHeaders,timeStamp,accept_language,user_guid,type,parentFrameId,windowName,statusCode,documentReferer,x_forwarded_for,user_agent,request_unixtime,country_code,user_map,requestType,method,partner_id,frameId,requestId,mainFrameRequestId,ip,filepath,redirectUrl,fromCache,url,software_id,statusLine,windowTitle,tabId,responseHeaders,openerTabId,request_datetime
0,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,sub_frame,0,None,200,None,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207176,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,3807,46079,None,54.239.31.89,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,http://s.amazon-adsystem.com/iu3?d=imdb.com&a1...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,None,335,"{'Vary': 'User-Agent', 'Date': 'Wed, 07 Sep 20...",None,2016-09-06 20:12:56
1,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,main_frame,-1,,200,http://www.imdb.com/title/tt0364845/episodes?r...,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207176,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,0,46030,None,207.171.162.180,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,http://www.imdb.com/title/tt0364845/eprate?ref...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,"""NCIS"" (2003)",335,"{'Date': 'Wed, 07 Sep 2016 00:12:37 GMT', 'Tra...",None,2016-09-06 20:12:56
2,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,sub_frame,0,None,200,None,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207159,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,3799,46001,None,31.13.71.36,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,https://www.facebook.com/widgets/like.php?widt...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200,None,335,"{'content-type': 'text/html', 'status': '200',...",None,2016-09-06 20:12:39
3,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,sub_frame,0,None,200,None,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207159,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,3801,46002,None,54.239.31.89,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,http://s.amazon-adsystem.com/iu3?d=imdb.com&a1...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,None,335,"{'Vary': 'User-Agent', 'Date': 'Wed, 07 Sep 20...",None,2016-09-06 20:12:39
4,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,main_frame,-1,,200,http://www.imdb.com/title/tt0364845/?ref_=nv_sr_1,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207159,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,0,45987,None,207.171.162.180,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,True,http://www.imdb.com/title/tt0364845/episodes?r...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,NCIS (TV Series 2003â ) - Episodes - IMDb,335,"{'Content-Language': 'en-US', 'Date': 'Wed, 07...",None,2016-09-06 20:12:39


In [7]:
gzip_paths = get_gzip_file_paths(root_directory)
clickthrough_dfs = read_gzip_files(gzip_paths[:2], swap_directory, failed_file_name)

# Using all headers for now. Later depending on the usecase, we can use a subset of these
# columns for our analysis
headers = get_all_headers(clickthrough_dfs)
clickthrough_dfs = [format_clickthrough_dataframes(df, headers) for df in clickthrough_dfs]
input_data = pd.concat(clickthrough_dfs)

make_readable_date(input_data)
input_data.head()

205it [00:00, 30904.76it/s]


,requestHeaders,timeStamp,accept_language,user_guid,type,parentFrameId,windowName,statusCode,documentReferer,x_forwarded_for,user_agent,request_unixtime,country_code,user_map,requestType,method,partner_id,frameId,requestId,mainFrameRequestId,ip,filepath,redirectUrl,fromCache,url,software_id,statusLine,windowTitle,tabId,responseHeaders,openerTabId,request_datetime
0,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,sub_frame,0,None,200,None,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207176,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,3807,46079,None,54.239.31.89,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,http://s.amazon-adsystem.com/iu3?d=imdb.com&a1...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,None,335,"{'Vary': 'User-Agent', 'Date': 'Wed, 07 Sep 20...",None,2016-09-06 20:12:56
1,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,main_frame,-1,,200,http://www.imdb.com/title/tt0364845/episodes?r...,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207176,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,0,46030,None,207.171.162.180,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,http://www.imdb.com/title/tt0364845/eprate?ref...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,"""NCIS"" (2003)",335,"{'Date': 'Wed, 07 Sep 2016 00:12:37 GMT', 'Tra...",None,2016-09-06 20:12:56
2,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,sub_frame,0,None,200,None,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207159,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,3799,46001,None,31.13.71.36,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,https://www.facebook.com/widgets/like.php?widt...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200,None,335,"{'content-type': 'text/html', 'status': '200',...",None,2016-09-06 20:12:39
3,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,sub_frame,0,None,200,None,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207159,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,3801,46002,None,54.239.31.89,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,False,http://s.amazon-adsystem.com/iu3?d=imdb.com&a1...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,None,335,"{'Vary': 'User-Agent', 'Date': 'Wed, 07 Sep 20...",None,2016-09-06 20:12:39
4,"{'Upgrade-Insecure-Requests': '1', 'Accept-Lan...",1.473207e+12,"en-US,en;q=0.8",2f8b23ca273de94a51281b0697a126d7,main_frame,-1,,200,http://www.imdb.com/title/tt0364845/?ref_=nv_sr_1,69.207.104.248,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1473207159,US,{'tiger': '0b3cefeb-182c-9f91-b3ae-836baaa448c8'},main,GET,1697b2090acdf06d,0,45987,None,207.171.162.180,09/07/00/US/CR/1697b2090acdf06d/part-00014-000...,None,True,http://www.imdb.com/title/tt0364845/episodes?r...,d2c091f86191954cdf6e24beb1d2092a,HTTP/1.1 200 OK,NCIS (TV Series 2003â ) - Episodes - IMDb,335,"{'Content-Language': 'en-US', 'Date': 'Wed, 07...",None,2016-09-06 20:12:39


In [13]:
input_data.iloc[45]["user_agent"]

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'

In [9]:
gp1 = input_data.groupby("user_guid")["software_id"].nunique().reset_index()

In [11]:
gp1["software_id"].unique()

array([1])

##### 0. Has gzip files been corrupted
    1. No empty files
    2. Has all mandatory columns
    3. See if the filepath url is valid
    4. check for a filter condition

# Make sure that all columns exist in all the dataframes

#### Profile this data (pandas profiling works better here, unfortunately a system error stops me from doing this)